<a href="https://colab.research.google.com/github/nrvanwyck/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data -o voting_data.csv

import pandas as pd

# data set has no header; this will fix that
column_headers = ['political-party', 'handicapped-infants', 
                  'water-project-cost-sharing', 
                  'adoption-of-the-budget-resolution', 'physician-fee-freeze', 
                  'el-salvador-aid', 'religious-groups-in-schools',
                  'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 
                  'mx-missile', 'immigration', 'synfuels-corporation-cutback', 
                  'education-spending', 'superfund-right-to-sue', 'crime', 
                  'duty-free-exports', 
                  'export-administration-act-south-africa']

# NaNs stored as '?'s; we will make them NaNs
voting_data = pd.read_csv('voting_data.csv', names=column_headers, 
                          na_values='?')

# let's turn all the yeses and nos to 1s and 0s:
voting_data.replace({'y': 1, 'n': 0}, inplace=True)

voting_data.head()
# looks good

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18171  100 18171    0     0  31114      0 --:--:-- --:--:-- --:--:-- 31061


,political-party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
from scipy.stats import ttest_ind

In [0]:
rep = voting_data[voting_data['political-party'] == 'republican']
dem = voting_data[voting_data['political-party'] == 'democrat']

print("handicapped-infants ttest: ")
print(ttest_ind(dem['handicapped-infants'], rep['handicapped-infants'], 
          nan_policy='omit'))

print("\nwater-project-cost-sharing ttest: ")
print(ttest_ind(dem['water-project-cost-sharing'], 
          rep['water-project-cost-sharing'], nan_policy='omit'))

print("\nadoption-of-the-budget-resolution ttest: ")
print(ttest_ind(dem['adoption-of-the-budget-resolution'], 
          rep['adoption-of-the-budget-resolution'], nan_policy='omit'))

print("\nphysician-fee-freeze ttest: ")
print(ttest_ind(dem['physician-fee-freeze'], rep['physician-fee-freeze'], 
          nan_policy='omit'))

print("\nel-salvador-aid ttest: ")
print(ttest_ind(dem['el-salvador-aid'], rep['el-salvador-aid'], nan_policy='omit'))

print("\nreligious-groups-in-schools ttest: ")
print(ttest_ind(dem['religious-groups-in-schools'], 
          rep['religious-groups-in-schools'], nan_policy='omit'))

print("\nanti-satellite-test-ban ttest: ")
print(ttest_ind(dem['anti-satellite-test-ban'], rep['anti-satellite-test-ban'], 
          nan_policy='omit'))

print("\naid-to-nicaraguan-contras ttest: ")
print(ttest_ind(dem['aid-to-nicaraguan-contras'], rep['aid-to-nicaraguan-contras'], 
          nan_policy='omit'))

print("\nmx-missile ttest: ")
print(ttest_ind(dem['mx-missile'], rep['mx-missile'], nan_policy='omit'))

print("\nimmigration ttest: ")
print(ttest_ind(dem['immigration'], rep['immigration'], nan_policy='omit'))

print("\nsynfuels-corporation-cutback ttest: ")
print(ttest_ind(dem['synfuels-corporation-cutback'],
          rep['synfuels-corporation-cutback'], nan_policy='omit'))

print("\neducation-spending ttest: ")
print(ttest_ind(dem['education-spending'], rep['education-spending'], 
          nan_policy='omit'))

print("\nsuperfund-right-to-sue ttest: ")
print(ttest_ind(dem['superfund-right-to-sue'], rep['superfund-right-to-sue'], 
          nan_policy='omit'))

print("\ncrime ttest: ")
print(ttest_ind(dem['crime'], rep['crime'], nan_policy='omit'))

print("\nduty-free-exports ttest: ")
print(ttest_ind(dem['duty-free-exports'], rep['duty-free-exports'], 
          nan_policy='omit'))

print("\nexport-administration-act-south-africa ttest: ")
print(ttest_ind(dem['export-administration-act-south-africa'], 
          rep['export-administration-act-south-africa'], nan_policy='omit'))

handicapped-infants ttest: 
Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)

water-project-cost-sharing ttest: 
Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

adoption-of-the-budget-resolution ttest: 
Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

physician-fee-freeze ttest: 
Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)

el-salvador-aid ttest: 
Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68)

religious-groups-in-schools ttest: 
Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)

anti-satellite-test-ban ttest: 
Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31)

aid-to-nicaraguan-contras ttest: 
Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)

mx-missile ttest: 
Ttest_indResult(statistic=16.437503268542994, pvalue=5.03079265310811e-47)

immigration ttest: 
Ttest_indR

Democrats were more likely to support the bills on:

- handicapped-infants
- adoption-of-the-budget-resolution
- anti-satellite-test-ban
- aid-to-nicaraguan-contras
- mx-missile
- synfuels-corporation-cutback
- duty-free-exports
- export-administration-act-south-africa

with a p-value of less than .01.

Republicans were more likely to support the bills on:

- physician-fee-freeze
- el-salvador-aid
- religious-groups-in-schools
- education-spending
- superfund-right-to-sue
- crime

with a p-value of less than .01.

There is not that much of a difference between Democrats and Republicans on:

- water-project-cost-sharing

When we ran t-tests on the above issue, the p-value was greater than .1 (it was .93).

The one issue not responsive to the three questions is:

- immigration

When we ran a t-test on the above issue, the p-value was greater than .01 but less than .1 (it was .08).